In [1]:
import nltk
import time
import numpy as np
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\senyo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Now we load the json

In [2]:
document_path = r"../../data/fashion_products_dataset.json"

import sys
from pathlib import Path
project_root = Path().resolve().parent.parent
sys.path.append(str(project_root))
from myapp.search.load_corpus import load_corpus
docs = load_corpus(document_path)

In [3]:
display(docs["TKPFCZ9EA7H5FYZH"])

Document(pid='TKPFCZ9EA7H5FYZH', title='Solid Women Multicolor Track Pants', description='Yorker trackpants made from 100% rich combed cotton giving it a rich look.Designed for Comfort,Skin friendly fabric,itch-free waistband & great for all year round use Proudly made in India', brand='York', category='Clothing and Accessories', sub_category='Bottomwear', product_details={'Style Code': '1005COMBO2', 'Closure': 'Elastic', 'Pockets': 'Side Pockets', 'Fabric': 'Cotton Blend', 'Pattern': 'Solid', 'Color': 'Multicolor'}, seller='Shyam Enterprises', out_of_stock=False, selling_price=921.0, discount=69.0, actual_price=2999.0, average_rating=3.9, url='https://www.flipkart.com/yorker-solid-men-multicolor-track-pants/p/itmd2c76aadce459?pid=TKPFCZ9EA7H5FYZH&lid=LSTTKPFCZ9EA7H5FYZHVYXWP0&marketplace=FLIPKART&srno=b_1_1&otracker=browse&fm=organic&iid=177a46eb-d053-4732-b3de-fcad6ff59cbd.TKPFCZ9EA7H5FYZH.SEARCH&ssid=utkd4t3gb40000001612415717799', images=['https://rukminim1.flixcart.com/image/128/1

In [4]:
display(docs["TKPFCZ9EA7H5FYZH"].title)

'Solid Women Multicolor Track Pants'

In [5]:
print("Total documents:", len(docs))

Total documents: 28080


In [7]:
from myapp.search.algorithms import create_index_tfidf, compute_line_docs
docs = compute_line_docs(docs)
start_time = time.time()
num_documents = len(docs)
index, tf, df, idf, title_index = create_index_tfidf(docs, num_documents)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

TypeError: sequence item 5: expected str instance, dict found

In [ ]:
from myapp.search.algorithms import search_tfidf
print("Insert your query (i.e.: Computer Science):\n")
query = input()
ranked_docs = search_tfidf(query, index, idf, tf, title_index)
top = 10

print("\n======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
for d_id in ranked_docs[:top]:
    print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))

# Exploratory Data Analysis

In [ ]:
title_word_count = 0
descriptions_word_count = 0
sell_price = 0
actual_price = 0
for doc in docs.values():
    if getattr(doc, "title", None):
        title_word_count += len(doc.title.split())
    if getattr(doc, "description", None):
        descriptions_word_count += len(doc.description.split())
    if getattr(doc, "selling_price", None):
        sell_price += float(doc.selling_price)
    if getattr(doc, "actual_price", None):
        actual_price += float(doc.actual_price)

average_title_length = title_word_count / len(docs)
print("Average title length (in words):", np.round(average_title_length, 2))

average_description_length = descriptions_word_count / len(docs)
print("Average description length (in words):", np.round(average_description_length, 2))

average_selling_price = sell_price / len(docs)
print("Average selling price:", np.round(average_selling_price, 2))

average_actual_price = actual_price / len(docs)
print("Average actual price:", np.round(average_actual_price, 2))

Average title length (in words): 6.46
Average description length (in words): 29.77
Average selling price: 705.58
Average actual price: 1415.25
